In [4]:
import re
import os
import sys
import subprocess
import importlib
import requests
import urllib3

# --- 0. 환경 설정 및 라이브러리 로드 ---
def install_and_import(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"필수 라이브러리({package}) 설치 중...")
        # pywin32는 pypiwin32로 설치해야 함
        pkg_name = "pypiwin32" if package == "win32com" else package
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name, "--user"])
        importlib.invalidate_caches()

# 필수 라이브러리 설치
install_and_import("deep_translator")
install_and_import("win32com") # pywin32

from deep_translator import GoogleTranslator
import win32com.client as win32

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- Word (.doc) 파일 읽기/쓰기 함수 ---
def read_doc(path):
    abs_path = os.path.abspath(path)
    word = win32.gencache.EnsureDispatch('Word.Application')
    word.Visible = False
    doc = word.Documents.Open(abs_path)
    text = doc.Content.Text
    doc.Close()
    # word.Quit() # 여러 파일을 처리할 경우 마지막에 호출하는 것이 좋음
    return text.replace('\r', '\n')

def save_doc(text, path):
    abs_path = os.path.abspath(path)
    word = win32.gencache.EnsureDispatch('Word.Application')
    word.Visible = False
    doc = word.Documents.Add()
    doc.Content.Text = text
    # wdFormatDocument (0) 은 .doc 형식 포맷임
    doc.SaveAs(abs_path, FileFormat=0) 
    doc.Close()
    word.Quit()

# --- I. 숫자 시퀀스 괄호 처리 로직 ---
def enclose_numbers_in_parentheses(text):
    pattern = r"(\b(?!claims?\b)[a-zA-Z]+\b)(\s\d+[a-z0-9\-\']*(?:[\s,]+(?:and|or|to|-)\s+\d+[a-z0-9\-\']*(?!\w)|[\s,]+\d+[a-z0-9\-\']*(?!\w))*)"
    def replacement_logic(match):
        prefix = match.group(1)
        number_sequence = match.group(2).strip() 
        return f"{prefix}({number_sequence})"
    return re.sub(pattern, replacement_logic, text, flags=re.IGNORECASE)

# --- II. 영한 번역 로직 ---
def chunk_text(text, max_size=5000):
    chunks = []
    start = 0
    while start < len(text):
        lookahead_end = min(start + max_size, len(text))
        segment = text[start:lookahead_end]
        if lookahead_end == len(text):
            if segment.strip(): chunks.append(segment.strip())
            break
        best_split_index = -1
        for char in ['.', '?', '!']:
            last_punc_index = segment.rfind(char)
            if last_punc_index > best_split_index:
                best_split_index = last_punc_index
        if best_split_index != -1:
            best_end = start + best_split_index + 1
            chunks.append(text[start:best_end].strip())
            start = best_end
        else:
            chunks.append(segment.strip())
            start = lookahead_end
    return chunks

def translate_text(text):
    translator = GoogleTranslator(source='en', target='ko')
    chunks = chunk_text(text)
    translated_chunks = []
    for i, chunk in enumerate(chunks):
        print(f"   > 번역 진행 중... ({i+1}/{len(chunks)})")
        try:
            translated_chunks.append(translator.translate(chunk))
        except Exception as e:
            translated_chunks.append(f"[번역 오류: {e}]")
    return "\n\n".join(translated_chunks)

# --- III. 비격식체 변환 로직 ---
def transform_korean_sentence(text):
    conversion_rules = {
        "입니다": "이다", "합니다": "한다", "있습니다": "있다",
        "됩니다": "된다", "냅니다": "낸다", "시킵니다": "시킨다",
        "필요한다": "필요하다", "유리한다": "유리하다",
        "했습니다": "했다" , "줍니다": "준다",
        "없습니다": "없다", "않습니다": "않는다",
        "무화과.": "도", "무화과": "도", "그림": "도",
        "매개변수": "파라미터",  "종방향": "길이방향",
        "방사상으로": "반경방향으로"
    }
    for formal, casual in conversion_rules.items():
        text = text.replace(formal, casual)
    return text

# --- 메인 실행 프로세스 ---
def main():
    print("="*60)
    print(" 특허/기술 문서 Word 번역 프로세서 (.doc 전용)")
    print("="*60)

    input_path = input("✅ 원본 영문 .doc 파일 경로를 입력하세요: ").strip()
    if not os.path.exists(input_path):
        print("파일이 존재하지 않습니다.")
        return

    base_name = os.path.splitext(input_path)[0]
    final_path = input(f"✅ 결과 저장 경로 (엔터 시 '{base_name}_ko.doc'): ").strip()
    if not final_path:
        final_path = f"{base_name}_ko.doc"

    print("\n[STEP 1] .doc 파일 읽기 및 전처리 중...")
    en_text = read_doc(input_path)
    bracketed_text = enclose_numbers_in_parentheses(en_text)

    print("\n[STEP 2] 영한 번역 진행 중...")
    translated_text = translate_text(bracketed_text)

    print("\n[STEP 3] 문체 변환 및 .doc 저장 중...")
    final_text = transform_korean_sentence(translated_text)
    save_doc(final_text, final_path)
    
    print("\n" + "="*60)
    print(f"완료! 저장 위치: {final_path}")
    print("="*60)

if __name__ == "__main__":
    main()


필수 라이브러리(win32com) 설치 중...


ModuleNotFoundError: No module named 'pywintypes'

In [6]:
!pip uninstall pywin32 pypiwin32

^C


In [ ]:
!pip uninstall pywin32 pypiwin32

In [ ]:
!pip install pywin32